In [32]:
import pandas as pd
import numpy as np
import os

transferdir = "transfer/"

def process(text: str):
    from parse import compile
    from string import punctuation

    p = compile("{hit:d}.")
    in_list = text.split()
    out_list = list()
    for seg in in_list:
        parse_result = p.parse(seg)
        if parse_result:
            # We got a number with a dot afterward:
            out_list.append(seg.lower())
        else:
            out_list.append(seg.translate(str.maketrans("", "", punctuation)).lower())
    return " ".join(out_list)

df = pd.read_json("ParlaSpeech-HR.v1.0.jsonl", orient="records", lines=True)
df = df.rename(columns={"path":"hashname"})
df = df.loc[df.split.isin("test,dev".split(",")), :]

df["sentence"] = df.words.apply(" ".join).apply(process)
df["path"] = df.hashname.apply(lambda s: os.path.join(transferdir, s))

In [2]:

model_dir = "./38_with_lm/wav2vec2-xls-r-parlaspeech-hr/"
from transformers import AutoProcessor, Wav2Vec2ForCTC, AutoModelForCTC, Wav2Vec2ProcessorWithLM
processor = Wav2Vec2ProcessorWithLM.from_pretrained(model_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_dir)

Only 0 unigrams passed as vocabulary. Is this small or artificial data?


In [17]:

def get_transcript(audio_filepath:str):
    import soundfile as sf
    import torch

    speech, sample_rate = sf.read(audio_filepath)
    input_values = processor(speech, sampling_rate=sample_rate, return_tensors="pt").input_values.cuda()
    inputs = processor(speech, sampling_rate=sample_rate, return_tensors="pt")
    with torch.no_grad():
      logits = model(**inputs).logits
    transcription = processor.batch_decode(logits.numpy()).text

    return transcription[0]

In [18]:
from datasets import load_metric
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
from tqdm.auto import tqdm
df["predictions"] = [get_transcript(path) for path in tqdm(df.path.values)]
for splt in ["dev", "test"]:
    print("Evaluating on ", splt)
    wer = wer_metric.compute(
        references=df.loc[df.split==splt, "sentence"],
        predictions=df.loc[df.split==splt, "predictions"]
    )

    cer = cer_metric.compute(
        references=df.loc[df.split==splt, "sentence"],
        predictions=df.loc[df.split==splt, "predictions"]
    )

    print(f"{wer=:0.4f}, {cer=:0.4f}")

  0%|          | 0/1013 [00:00<?, ?it/s]

Evaluating on  dev
wer=0.1129, cer=0.0448
Evaluating on  test
wer=0.0985, cer=0.0363


In [22]:
df[["sentence", "predictions", "split", "hashname"]].to_csv(
    "39_model_with_LM_output_for_evaluation.csv", 
    index=False
    )

In [24]:
pd.set_option('display.max_colwidth', None)
df[["sentence", "predictions", "split", "hashname"]].sample(20)

,sentence,predictions,split,hashname
303677,počevši od 1. siječnja 2018. godine stupanjem na snagu pravilnika o elektroničkoj žalbi u javnoj nabavi gospodarski subjekti imaju mogućnost izjavljivanja žalbe putem međusobno povezanih informacijskih sustava državne komisije i elektroničkog oglasnika javne nabave primjena elektroničke žalbe dovela je do određenih pozitivnih,počevši od 1. siječnja 2018. godine stupanjem na snagu pravilnika o elektroničkoj žalbi u javnoj nabavi gospodarski subjekti imaju mogućnost izjavljivanja žalbe putem međusobno povezanih informacijskih sustava državne komisije i elektroničkog oglasnika javne nabave primjena elektroničke žalbe dovela je do određenih pozitivnih,test,PPYtWHK4KPk_5335.6-5355.1.wav
113278,pa evo ovako ja ni približno nemam namjeru iskoristiti 30 minuta za ovo izlaganje u ovo pozno doba i pred ovako prilično praznom sabornicom ali nekoliko komentara moram dati u smislu,evo ovako ja ni približno nemam namjeru iskoristiti 30 minuta za ovo izlaganje u ovo pozno doba i pred ovako prilično praznom sabornicom ali nekoliko komentara moram dati u smislu,test,dpKsbH3SErQ_17443.4-17461.56.wav
277021,povećanje broja korisnika doplatka za djecu približno do 50 odnosno 150000 novih potencijalnih korisnika istovremeno proširit će se i krug potencijalnih korisnika pronatalitetnog dodatka od 500 kuna koji ostvaruju korisnici doplatka za djecu,povećanje broja korisnika doplatka za djecu približno do 50 odnosno oko 150000 novih potencijalnih korisnika istovremeno proširit će se i krug potencijalnih korisnika pronatalitetnog dodatka od 500 kuna koji ostvaruju korisnici doplatka za djecu,test,Kjyn-8vCedk_692.81-712.33.wav
210924,28 žalbenih predmeta započeti su izjavljivanjem žalbe u obliku elektroničkim putem u obliku elektroničke žalbe,28 žalbenih predmeta započeti su izjavljivanjem žalbe u obliku elektroničkim putem u obliku elektroničke žalbe,test,PPYtWHK4KPk_5553.68-5561.4.wav
1290,u društvu koji su glupi a postoji oni koji se prave glupi ja mislim da je naš ministar marić prepametan da bi bio glup iz čega,u društvu koji su glupi a postoje i oni koji se prave glupi ja mislim da je naš ministar marić prepametan da bi bio glup iz čega,dev,f9bYjY87cMI_4494.59-4507.87.wav
105050,dokumenta kao sastavnice programa gradnje objekata i uređaja komunalne infrastrukture kojom bi se čak definirale i namjene sa osnova prihoda naknade za uređenje voda a koji bi se odnosio sukladno zakonu kojim se uređuje komunalno gospodarstvo napominje se da bi ovakav prijedlog doveo do neučinkovitosti odlučivanja,dokumenta kao sastavnice programa gradnje objekata i uređaja komunalne infrastrukture koji bi se čak definirale i namjene sa osnova prihoda naknade za uređenje voda a koji bi se odnosio sukladno zakonu kojim se uređuje komunalno gospodarstvo napominje se da bi ovakav prijedlog doveo do neučinkovitosti odlučivanja,test,x1XlcaDsSf8_8925.96-8945.94.wav
4032,dakle zakon o izvlaštenju itd itd što se pokazalo neispravnim jer sada polako kada smo vidjeli da nam taj zakon nije donio nikakve stvari poboljšice da ništa nije ubrzao da je eventualno nešto popisao ili napravio listu tih projekata ali se nije moglo ništa s time ubrzati jer niti jedan državni,dakle zakon o izvlaštenju itd itd što se pokazalo ispravnim jer sada polako kada smo vidjeli da nam taj zakon nije donio nikakve stvari poboljšice da ništa nije ubrzao da je eventualno nešto potpisao napravio je listu tih projekata ali se nije ništa s time ubrzati jer niti jedan državni,dev,Ux94YXzTmpo_12753.05-12772.84.wav
2409,to da hdz i plenković opstanu u poziciji na kojoj nažalost danas jesu i loše obavljaju svoj posao ono što mogu obećati da ova intencija i ovo zakonske rješenje će ostati na snazi i kada se promijeni plenkovićeva vlada bez,to da hdz i plenković opstanu u poziciji na kojoj nažalost danas jesu i loše obavljaju svoj posao ono što mogu obećati da ova intencija i ova zakonska rješenja će ostati na snazi i kada se promijeni plenkovićeva vlada bez,dev,-EO06cT21u

In [25]:
df["wer"] = df.apply(
    lambda row: wer_metric.compute(
        references = [row["sentence"]],
        predictions= [row["predictions"]]),
        axis=1
)

df["cer"] = df.apply(
    lambda row: cer_metric.compute(
        references = [row["sentence"]],
        predictions= [row["predictions"]]),
        axis=1
)

In [31]:
df.sort_values(by="cer", ascending=False)[["sentence", "predictions", "cer"]].head(20)

,sentence,predictions,cer
1545,iznio stav kluba hrvatske demokratske zajednice to ću kad budem imao prigode reći što mislim reći o,iznio stav kluba hdza to ću kad budem imao prigode reći što mislimo,0.323232
42061,ukupnom području republike hrvatske,ukupnom području rhatske,0.314286
75436,stupanja zakona na snagu a to je 14 ove godine sama procjena fiskalnog učinka ovog zakona na razini ove godine iznosi 160950000 kuna odnosno na stavkama hzzoa,stupanja zakona na snagu a to je 14 ove godine sama procjena fiskalnog učinka ovog zakona na razini ove godine iznosi 160 milijuna 950 tisuća kuna odnosno na stavkama hrvatskog zavoda za zdrav,0.240506
649,revident a revidenata ima malo pogotovo za pojedine oblasti,revidenata ima malo pogotovo za pojedine oblasti,0.186441
44917,86 slučajeva,86 slučaja,0.166667
70625,predsjednika republike f tuđmana,predsjednika republike franje tuđmana,0.156250
1354,kada se samo sjetimo onih tužnih brojki od prije tri četiri godine oni su za ove brojke sada dobre na žalost bile su ružne ali šta se sada dešava,kada se samo sjetimo onih tužnih brojki od prije 3 4 godine oni su za ove brojke sada izgledaju dobra nažalost bile su ružne ali šta se sada dešava,0.144828
491,niti ina niti kamioni nisu još bili se angažirali u slučaju jedne i druge afere ali to ovdje nije slučaj,niti ina niti kani oni nisu još bili se angažirali u slučaju jednog jedne druge afere ali ovdje to nije slučaj,0.144231
4627,ishođenje zadovoljštine zbog toga što su oklevetani ne može doprinijeti dignitetu hrvatskoga sabora stoga jest temeljna uloga predsjedavajućega predsjednika sabora predsjedavajućeg sabora i,ishođenje zadovoljštine zbog toga što su oklevetani ne može doprinijeti dignitetu hrvatskoga sabora stoga jest temeljna uloga predsjednika predsjedavajućeg sabora i,0.132275
246,posljednje vrijeme dakle pred 10ak godina smo neke stvari zvali problemom problematiko sad o tome kad govorimo govorimo o izazovima ono što jasno treba reći povećana je potrošnja,posljednje vrijeme dakle pred 10ak godina smo neke stvari zvali problemom problematikom sad to govorimo sada o tome kad govorimo govorimo o izazovima ono što jasno treba reći povećana potrošnja,0.117978
